In [1]:
import itertools as it
from collections import defaultdict, Counter
from functools import reduce
from operator import add
from math import factorial
from string import ascii_uppercase
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Generate Unique Paths for All Permutations of Matches

In [3]:
def gen_contestants(N):
    """
    Generate a unique set of girls and guys for N couples.
    Returns a dictionary of each set.
    """
    return dict(
        girls=set(it.islice(ascii_uppercase, N, 2 * N)),
        guys=set(it.islice(ascii_uppercase, 0, N))
    )

In [4]:
N = 4

def reduce_path(path):
    """
    Reduces a path of matches to an unnested set of individuals.
    """
    return reduce(add, path)


def unique_couples(path):
    """
    Tests whether a path of matches has unique individuals.
    """
    count = Counter(reduce_path(path)).values()
    return max(count) == 1


def generate_unique_paths(guys, girls):
    """
    Generate all unique paths (sets of couples).
    Returns a list of lists (paths) of tuples (couples).
    """
    # get all possible pairings
    girl_perms = it.permutations(girls, len(girls))
    return [list(zip(guys, girl_perm)) for girl_perm in girl_perms]

girls, guys = gen_contestants(N).values()
paths = generate_unique_paths(guys, girls)
print(f"Number of paths for {N} contestants: {len(paths):,.0f}")

n = factorial(N)
if n != len(paths):
    print(f"ERROR: {n} != {len(paths)}")

print("First 10 paths:")
for path in paths[:10]:
    print(path) #, unique_couples(path))

# Check that all paths have unique couples
# all([unique_couples(path) for path in paths])

Number of paths for 4 contestants: 24
First 10 paths:
[('D', 'F'), ('B', 'H'), ('A', 'G'), ('C', 'E')]
[('D', 'F'), ('B', 'H'), ('A', 'E'), ('C', 'G')]
[('D', 'F'), ('B', 'G'), ('A', 'H'), ('C', 'E')]
[('D', 'F'), ('B', 'G'), ('A', 'E'), ('C', 'H')]
[('D', 'F'), ('B', 'E'), ('A', 'H'), ('C', 'G')]
[('D', 'F'), ('B', 'E'), ('A', 'G'), ('C', 'H')]
[('D', 'H'), ('B', 'F'), ('A', 'G'), ('C', 'E')]
[('D', 'H'), ('B', 'F'), ('A', 'E'), ('C', 'G')]
[('D', 'H'), ('B', 'G'), ('A', 'F'), ('C', 'E')]
[('D', 'H'), ('B', 'G'), ('A', 'E'), ('C', 'F')]


### How many paths contain each of the matches?

In [5]:
def count_couple_paths(paths):
    """Count the number of paths that contain each couple."""
    n = len(paths)
    dd = defaultdict(int)
    for path in paths:
        for couple in path:
            dd[couple] += 1
    # sum((1 if ('A', 'K') in path else 0 for path in paths))/n
    df = pd.DataFrame.from_dict(dd, orient='index', columns=['count'])
    df['prob'] = df['count'] / n
    df.index.name = 'couple'

    return df.sort_values('count', ascending=False)

couple_paths = count_couple_paths(paths)
couple_paths.apply({'count': '{:,.0f}'.format,
                    'prob': '{:.1%}'.format}).head(12)

,count,prob
couple,,
"(D, F)",6,25.0%
"(B, H)",6,25.0%
"(A, G)",6,25.0%
"(C, E)",6,25.0%
"(A, E)",6,25.0%
"(C, G)",6,25.0%
"(B, G)",6,25.0%
"(A, H)",6,25.0%
"(C, H)",6,25.0%


## Example: Assume first match is in the first ceremony with > 0 and < 10 perfect matches
We know that at least one of the matches is correct, so any paths that do not contain any of those matches does not have any PM's.  Since we know that that matchup does not contain all 10 perfect matches, we can drop that one as a candidate.

In [6]:
def drop_paths_without_these_couples(paths, couples, inclusive=False):
    """
    Drop paths that do not contain any of the couples.

    Parameters
    ----------
    paths : list of lists
        List of paths (lists of tuples).
    couples : list of tuples
        A single path or List of 10 couples to drop.
    inclusive : bool, default False
        If True, drop the given path.

    Returns a list of paths that include at least
    one of the couples.
    """
    if inclusive:
        return set(filter(lambda x: any(v in x for v in couples) and \
                           x != couples, paths))
    else:
        return set(filter(lambda x: any(v in x for v in couples), paths))


def drop_path(paths, path):
    """
    Drop a path from a list of paths.
    """
    return set(filter(lambda x: x != path, paths))


def drop_paths_with_these_couples(paths, couples):
    """
    Drop paths that contain any of the couples.
    Returns a list of paths that do not contain
    any of the couples. Useful for "blackouts."
    """
    return set(filter(lambda x: not any(v in x for v in couples), paths))


def update_paths_for_ceremony(paths, ceremony_matchups, n_perfect_matches):
    """
    Update paths for a ceremony.
    Returns a list of paths that contain the ceremony.
    """
    n_matchups = len(ceremony_matchups)
    assert 0 <= n_perfect_matches <= n_matchups

    if n_perfect_matches == 0:
        # blackout
        return drop_paths_with_these_couples(paths, ceremony_matchups)
    elif n_perfect_matches == n_matchups:
        return ceremony_matchups
    else:
        return drop_paths_without_these_couples(paths, ceremony_matchups,
                                                inclusive=True)


ceremony1_matchup = paths[0]
print(f"Ceremony 1 matchup: {ceremony1_matchup}")
remaining1 = update_paths_for_ceremony(paths, ceremony1_matchup, 2)
n_paths = len(paths)
n_remaining1 = len(remaining1)
print(f"Number of paths dropped: {n_paths - n_remaining1:,.0f} leaving {n_remaining1:,.0f} paths")

Ceremony 1 matchup: [('D', 'F'), ('B', 'H'), ('A', 'G'), ('C', 'E')]


AssertionError: 

### Which paths *now* have the highest probabilities?

In [17]:
couple_paths1 = count_couple_paths(remaining1)
couple_paths1.apply({'count': '{:,.0f}'.format,
                     'prob': '{:.1%}'.format}).head(12)

,count,prob
couple,,
"(A, K)","362,879",15.8%
"(J, T)","362,879",15.8%
"(B, L)","362,879",15.8%
"(H, R)","362,879",15.8%
"(I, S)","362,879",15.8%
"(F, P)","362,879",15.8%
"(D, N)","362,879",15.8%
"(C, M)","362,879",15.8%
"(E, O)","362,879",15.8%


In [41]:
couple_paths1.iloc[0]

count    362879.000000
prob          0.158197
Name: (A, K), dtype: float64

In [40]:
362879/len(remaining1)

0.15819730948741803

Those that were dropped *increased* in probability (by 5.8%) because now there are relatively more paths that contain those matches.  Similarly, all other matches decreased in probability (by 0.6%) based on the number of paths containing those matches that are left.

**This, however, does not account for the actual number of PM's that's revealed at the ceremony.**

In [34]:
len(remaining1)

2293838

In [30]:
couple_paths1['count'].value_counts()

count
214551    90
362879    10
Name: count, dtype: int64

In [29]:
couple_paths1.loc[ceremony1_matchup, 'prob'] - .1

couple
(A, K)    0.058197
(B, L)    0.058197
(C, M)    0.058197
(D, N)    0.058197
(E, O)    0.058197
(F, P)    0.058197
(G, Q)    0.058197
(H, R)    0.058197
(I, S)    0.058197
(J, T)    0.058197
Name: prob, dtype: float64

In [27]:
cp1_prob = couple_paths1['prob'].to_dict()
remaining1_probs = pd.Series([sum(cp1_prob[couple] for couple in path)
                              for path in remaining1], name='path_prob',
                              dtype=np.float32).sort_values(ascending=False)
remaining1_probs.value_counts().head(10)


path_prob
1.000000    1334960
1.064664     667485
1.129327     222480
1.193991      55650
1.258655      11088
1.323318       1890
1.387982        240
1.452646         45
Name: count, dtype: int64

In [ ]:
top_paths_idx = remaining1_probs.index[:10]
top_paths = [remaining1[i] for i in top_paths_idx]
top_paths_midx = pd.MultiIndex.from_tuples([top_paths, range(1, 11)], names=['couple', ''])
top_paths_probs = remaining1_probs[top_paths_idx]

## Example: Truth Booth says (A, L) is not a Perfect Match

In [6]:
p2d = filter(lambda x: ('A', 'L') in x, remaining1)
n_p2d = len(list(p2d))
remaining2 = [path for path in remaining1 if ('A', 'L') not in path]
print(f"Number of paths with A-L: {n_p2d:,.0f}, leaving {len(remaining1) - n_p2d:,.0f} paths")
print(f"check: {len(remaining2):,.0f} paths")

Number of paths with A-L: 214,551, leaving 2,079,288 paths
check: 2,079,288 paths


In [7]:
n = len(remaining2)
c = Counter((match for path in remaining2 for match in path))
probs = {k: v / n for k, v in c.items()}
freqs = Counter(probs.values())
freqs

Counter({0.0928380291715241: 56,
         0.0918189303261501: 16,
         0.10318484019529763: 16,
         0.15513002527788358: 8,
         0.17452127843761903: 2,
         0.09092727895318013: 1})

In [9]:
np.average(list(freqs.keys()), weights=list(freqs.values()))

0.10101010101010101

# OLD CODE ...

In [10]:
N = 3
guys = list('ABC')
girls = list('FGH')
perms = it.permutations(girls, N)

matches = {tuple(p): 1
           for perm in perms
           for p in zip(guys, perm)}
matches

{('A', 'F'): 1,
 ('B', 'G'): 1,
 ('C', 'H'): 1,
 ('B', 'H'): 1,
 ('C', 'G'): 1,
 ('A', 'G'): 1,
 ('B', 'F'): 1,
 ('C', 'F'): 1,
 ('A', 'H'): 1}

In [11]:
for (i, c), v in matches.items():
    print(i, c, v)

A F 1
B G 1
C H 1
B H 1
C G 1
A G 1
B F 1
C F 1
A H 1


In [12]:
N = 10
guys = range(N)
girls = range(N)
perms = it.permutations(girls, N)
    
paths = [list(zip(guys, perm)) for perm in perms]
n = len(paths)
print(f'{n:,.0f} paths')

3,628,800 paths


In [13]:
idx = pd.Index(range(10), name='Guys')
cols = pd.Index(range(10), name='Girls')

In [14]:
path = paths[0]
path

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9)]

In [15]:
def gen_matrix(males, females):
    z = np.zeros((len(males), len(females)),
                 dtype='int')
    z[males, females] = 1
    return z

perms = it.permutations(girls, N)
mats = np.concatenate([gen_matrix(guys, g) for g in perms])
mats.shape

(36288000, 10)

In [16]:
n = mats.shape[0]
idx = pd.Index(list(range(N)) * int(n/N), dtype='int', name='Guys')
M = pd.DataFrame(mats, index=idx, columns=cols)
data = np.array([i for i in range(int(n/N)) for _ in range(N)],
                dtype='int')
M['Iteration'] = data
M = M.set_index('Iteration', append=True)
M.head(15)

,Girls,0,1,2,3,4,5,6,7,8,9
Guys,Iteration,,,,,,,,,,
0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,0,0,0,1,0


#### Check for dupes

In [17]:
M.reset_index().duplicated().sum()

In [18]:
for path in paths[:5]:
    print(path)

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 7), (8, 9), (9, 8)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 8), (8, 7), (9, 9)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 8), (8, 9), (9, 7)]
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (5, 5), (6, 6), (7, 9), (8, 7), (9, 8)]


In [19]:
tots = M.groupby(level='Iteration').sum()
tots.describe()

Girls,0,1,2,3,4,5,6,7,8,9
count,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0,3628800.0
mean,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [20]:
tots.sum(1).value_counts()

10    3628800
dtype: int64

In [21]:
all = slice(None)
M.loc[(3, all), 4]

Guys  Iteration
3     0            0
      1            0
      2            0
      3            0
      4            0
                  ..
      3628795      0
      3628796      0
      3628797      0
      3628798      0
      3628799      0
Name: 4, Length: 3628800, dtype: int64

### Now that we have all possible matches, what is the best structure for working with them?
* A single matrix of all scenarios?
* A series of matrices?

### Need to be able to drop paths with invalid matches
* Could use brute force method and just search all paths for specific matches.  Is there a better way?

### How will you account for overall results?
* On the first round, all paths have equal probability and hence are equally weighted.
* On subsequent rounds, you will update the weights based on the results for that round.

In [22]:
import are_you_the_one as ayto

In [23]:
guys = [ayto.Guy(letter) for letter in list('ABCDE')]
girls = [ayto.Girl(letter) for letter in list('FGHIJ')]
tournament = ayto.Tournament(guys, girls)

In [24]:
tournament.grid.X

,F,G,H,I,J
A,0,0,0,0,0
B,0,0,0,0,0
C,0,0,0,0,0
D,0,0,0,0,0
E,0,0,0,0,0
